# Глава 7 Функции(Python Дэвид Бизли)

Определение функций с  помощью инструкции def – краеугольный камень всех программ. 

Цель этой главы состоит в том, чтобы представить несколько продвинутых и необычных определений функций и паттернов их использования. 

Темы рассматривают аргументы по умолчанию, функции, способные принимать любое
количество аргументов или только именованные аргументы, аннотации и  замыкания. 

Также мы рассмотрим несколько непростых задач контроля потока управления и передачи данных, которые относятся к функциям обратного вызова
(callback functions).


# 7.1. Определение функций, принимающих любое количество аргументов

Вы хотите определить функцию, которая принимает любое количество аргументов.

Чтобы определить функцию, которая принимает любое количество позиционных
аргументов, используйте аргумент со звездочкой (*argument):

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))
# Пример использования
avg(1, 2) # 1.5

1.5

In [2]:
avg(1, 2, 3, 4) # 2.5

2.5

В этом примере rest – это кортеж всех дополнительных позиционных аргументов. Код обрабатывает его как последовательность при выполнении вычислений.

Чтобы принять любое количество именованных аргументов, используйте аргумент, который начинается с **. Например:

In [3]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(name=name, 
                                                       attrs=attr_str, 
                                                       value=html.escape(value))
    return element


# Пример
# Создает '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)


'<item size="large" quantity="6">Albatross</item>'

In [4]:
# Создает '<p>&lt;spam&gt;</p>'
make_element('p', '<spam>')

'<p>&lt;spam&gt;</p>'

Здесь attrs – это словарь, который хранит переданные именованные аргументы
(если они были предоставлены).

Если вам нужна функция, которая может принимать и любое количество позиционных, и любое количество именованных аргументов, используйте * и ** вместе. Например:

In [6]:
def anyargs(*args, **kwargs):
    print(args) # Кортеж
    print(kwargs) # Словарь
    
anyargs('one', two=2, three=3)

('one',)
{'two': 2, 'three': 3}


В этой функции позиционные аргументы попадают в кортеж args, а все именованные аргументы – в словарь kwargs.

Аргумент со * может быть только последним в списке позиционных аргументов
в определении функции. 

Аргумент с ** может быть только последним.

Тонкость тут в том, что аргумент без звездочки может идти и после аргумента со звездочкой:

In [12]:
def a(x, *args, y):
    print(x, args, y)
a(1, 2, 3, y=4)

1 (2, 3) 4


In [15]:
def b(x, *args, y, **kwargs):
    print(x, args, y, kwargs)
b(1, 2, 3, y=4, five=5, six=6)

1 (2, 3) 4 {'five': 5, 'six': 6}


Такие аргументы известны как «обязательные именованные аргументы», они
обсуждаются далее в рецепте 7.2.

# 7.2. Определение функций, принимающих только именованные аргументы


Вам нужна функция, которая принимает только именованные аргументы.

Эту возможность легко реализовать, если вы поместите именованные аргументы
после аргумента со звездочкой или символа звездочки. Например:

In [18]:
def recv(maxsize, *, block):
    'Receives a message'
    print(maxsize, block)

recv(1024, True) # TypeError

TypeError: recv() takes 1 positional argument but 2 were given

In [19]:
recv(1024, block=True) # Ok

1024 True


Этот прием может быть также использован для определения именованных 
аргументов в функциях, которые принимают различное количество позиционных
аргументов. Например:

In [21]:
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

minimum(1, 5, 2, -5, 10) # Вернет -5

-5

In [22]:
minimum(1, 5, 2, -5, 10, clip=0) # Вернет 0

0

Обязательные именованные аргументы часто являются хорошим способом увеличить понятность кода при определении необязательных аргументов. 

Например, посмотрите на такой вызов:

In [23]:
msg = recv(1024, False)

TypeError: recv() takes 1 positional argument but 2 were given

Пользователь, который незнаком с  функцией recv, не имеет представления о том, что означает аргумент False. 

С другой стороны, такой вызов будет намного
более ясным:

In [25]:
msg = recv(1024, block=False)

1024 False


Использование обязательных именованных аргументов часто предпочтительнее трюков с  применением **kwargs, поскольку они правильно показываются,
когда пользователь просит помощи:

In [26]:
help(recv)
#Help on function recv in module __main__:
#recv(maxsize, *, block)
#Receives a message


Help on function recv in module __main__:

recv(maxsize, *, block)
    Receives a message



Обязательные именованные аргументы также полезны в  более продвинутых применениях. 

Например, они могут быть использованы для внедрения аргументов в функции, которые применяют правила использования *args и **kwargs для
получения всех входных параметров. 
См. рецепт 9.11.

# 7.3. Прикрепление информационных метаданных к аргументам функций

Вы определили функцию, но хотели бы прикрепить дополнительную информацию к аргументам, чтобы другим людям было легче понять, что делает эта функция.

Аннотации аргументов могут быть полезны, чтобы помочь программистам разобраться в том, как нужно применять функцию. 

Например, рассмотрим такую аннотированную функцию:

In [1]:
def add(x:int, y:int) -> int:
    return x + y

In [2]:
add(1, 2)

3

Интерпретатор Python не прикрепляет никакого семантического смысла к аннотациям.

Это не проверки типов, они вообще никак не влияют на поведение Python. 

Однако они могут помогать другим людям читать исходный код и понимать, что вы имели в виду. 

А вот сторонние инструменты и фреймворки могут прикреплять к  аннотациям семантический смысл. 

Также они появляются в документации:


In [3]:
help(add)
#Help on function add in module __main__:
#add(x: int, y: int) -> int

Help on function add in module __main__:

add(x: int, y: int) -> int



Хотя вы можете прикрепить любой объект к  функции в  качестве аннотации (например, числа, строки, экземпляры и т. д.), использование классов или строк имеет наибольший смысл.

Аннотации функции хранятся в атрибуте функции __annotations__. Например:

In [4]:
add.__annotations__
#{'y': <class 'int'>, 'return': <class 'int'>, 'x': <class 'int'>}

{'x': int, 'y': int, 'return': int}

Хотя можно придумать немало потенциальных применений аннотаций, обычно их используют для документации. 

Поскольку в Python нет объявлений типов, часто бывает сложно понять, что нужно передавать в функцию, когда вы просто читаете исходный код. 

Аннотации дают дополнительную информацию.

См. продвинутый пример в рецепте 9.20, который показывает, как использовать аннотации для реализации множественной диспетчеризации (т. е. перегруженных функций).

# 7.4. Возвращение функцией нескольких значений

Вы хотите, чтобы функция возвращала несколько значений.

Чтобы вернуть несколько значений из функции, просто сделайте возвращаемым значением кортеж. 

Например:

In [5]:
def myfun():
    return 1, 2, 3

In [6]:
a, b, c = myfun()

In [7]:
a

1

In [8]:
b

2

In [9]:
c

3

Хотя это выглядит так, будто myfun() возвращает несколько значений, на самом
деле создается кортеж. 

Это кажется немного замысловатым, но дело в том, что
кортеж задается не скобками, а запятыми. 

Например:

In [10]:
a = (1, 2) # Со скобками
a

(1, 2)

In [11]:
b = 1, 2 # Без скобок
b

(1, 2)

При вызове функций, которые возвращают кортеж, часто результат присваивают нескольким переменным. 

Это просто распаковка кортежа, описанная в рецепте 1.1. Возвращаемое значение также может быть присвоено одной переменной:

In [12]:
x = myfun()
x

(1, 2, 3)

# 7.5. Определение функций с аргументами по умолчанию

Вы хотите определить функцию или метод, где один или более аргументов являются необязательными и имеют значение по умолчанию.

In [1]:
def spam(a, b=42):
    print(a, b)
    
spam(1) # Ok. a=1, b=42

1 42


In [2]:
spam(1, 2) # Ok. a=1, b=2

1 2


Если значение по умолчанию – это изменяемый (мутабельный) контейнер, такой как список, множество или словарь, используйте None в качестве значения по
умолчанию:

In [ ]:
# Использование списка в качестве значения по умолчанию
def spam(a, b=None):
    if b is None:
        b = []


Если вместо предоставления значения по умолчанию вы хотите написать код, который просто проверяет, передано ли в необязательном аргументе целевое значение, используйте такую идиому:

In [9]:
_no_value = object()

def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')
    else:
        print('b =', b)


Вот как эта функция себя ведет:


In [7]:
spam(1)

No b value supplied


In [10]:
spam(1, 2) # b = 2

b = 2


In [11]:
spam(1, None) # b = None

b = None


Понаблюдайте за разницей между отсутствием переданного значения и передачей значения None.

Определение функций с аргументами по умолчанию – несложное дело, но не без тонкостей.

Во-первых, значения, назначенные значениями по умолчанию, связываются только один раз, во время определения функции. 

Попробуйте поэкспериментировать:

In [12]:
x = 42
def spam(a, b=x):
    print(a, b)
    
spam(1) # Эффекта нет

1 42


In [13]:
x = 23
spam(1)

1 42


Заметьте, как изменение переменной x (которая была использована в качестве
значения по умолчанию) не оказывает влияния на последующие события.

Во-вторых, значения, назначенные значениями по умолчанию, всегда должны
быть неизменяемыми объектами, такими как None, True, False, числа или строки.
Никогда не пишите такой код:


In [ ]:
def spam(a, b=[]): # НЕТ!


Если вы это сделаете, то столкнетесь со всеми возможными неприятностями, если значение по умолчанию когда-либо покинет пределы функции и будет изменено. 

Такие изменения навсегда поменяют значение по умолчанию и подействуют на все будущие вызовы функции. 

Например:

In [14]:
def spam(a, b=[]):
    print(b)
    return b

x = spam(1)
x

[]


[]

In [15]:
x.append(99)
x.append('Yow!')
x

[99, 'Yow!']

In [16]:
spam(1) # Возвращается измененный список!

[99, 'Yow!']


[99, 'Yow!']

Вероятно, вы хотели не этого. Чтобы избежать таких проблем, лучше назначить в качестве значения по умолчанию None и проверить его затем в функции, как показано в решении.

Использование оператора is при проверке None – важнейшая часть этого рецепта. 

Некоторые делают такую ошибку:

In [17]:
def spam(a, b=None):
    if not b: # НЕТ! Вместо этого используйте 'b is None'
        b = []

In [ ]:
def spam(a, b=None):
    if b is None: # Да! Правильно!
        b = []

Хотя None выдает значение False, многие другие объекты (например, строки нулевой длины, пустые списки, кортежи и словари) ведут себя так же. 

Так что показанная выше проверка будет ошибочно считать некоторые входные значения
отсутствующими. Например:

In [18]:
spam(1)

In [19]:
x = []

In [20]:
spam(1, x)

In [21]:
spam(1, 0)

In [22]:
spam(1, '')

In [ ]:
spam(1) # ОК
x = []
spam(1, x) # Невидимая ошибка. Значение x перезаписывается по умолчанию
spam(1, 0) # Невидимая ошибка. 0 игнорируется
spam(1, '') # Невидимая ошибка. '' игнорируется

Последняя часть этого рецепта – это особенно тонкий момент: функция, которая выполняет проверку, передано ли значение (любое) в необязательном аргументе. 

Хитрость в том, что вы не можете использовать None, 0 или False в качестве значения по умолчанию при проверке присутствия предоставленного пользователем аргумента (поскольку все они являются вполне допустимыми аргументами
и пользователь может передать их в функцию). 

Так что вам нужно делать проверку как-то по-другому.
Чтобы решить эту проблему, вы можете создать уникальный частный экземпляр object, как показано в решении (переменная _no_value). 

Затем вы проверяете предоставленный аргумент в функции, сравнивая его с этим специальным зна-чением, чтобы узнать, передан аргумент или нет. Идея в том, что крайне маловероятно, что пользователь передаст в  качестве входного значения экземпляр
_no_value. 

Поэтому это безопасное значение для проверки того, предоставлен ли экземпляр.


Использование object() может показаться необычным. 

object – это класс, который является обычным базовым классом (суперклассом) практически всех объектов Python. 

Вы можете создавать экземпляры object, но они не особенно интересны, поскольку не имеют каких-то полезных методов или атрибутов (в них нет словаря экземпляра, так что вы не можете присвоить им атрибуты). 

В общем-то, проверка идентичности – единственная вещь, для которой они полезны. 

Их можно использовать в качестве специальных значений, как и показано в вышеописанном решении.

# 7.6. Определение анонимных функций или встроенных функций (inline)

Вам нужно предоставить короткую функцию обратного вызова для использования в операции типа sort(), но вы не хотите определять отдельную однострочную функцию с  помощью инструкции def. 

Вместо этого вам бы пригодился способ
определить функцию «в строке».


Простые функции, которые просто вычисляют результат выражения, могут быть заменены инструкцией lambda. Например:

In [1]:
add = lambda x, y: x + y
add(2,3)

5

In [2]:
add('hello', 'world')

'helloworld'

Использование lambda абсолютно равноценно такому примеру:

In [3]:
def add(x, y):
    return x + y

add(2,3)

5

Обычно lambda используется в контексте какой-то другой операции, такой как сортировка или свертка (reduction) данных:

In [4]:
names = ['David Beazley', 
         'Brian Jones', 
         'Raymond Hettinger', 
         'Ned Batchelder']
sorted(names, key=lambda name: name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

Хотя lambda позволяет определить простую функцию, ее возможности сильно ограничены. 

В  частности, может быть определено только одно выражение, результат которого станет возвращаемым значением. 

Это значит, что никакие другие возможности языка, в  т. ч. множественные инструкции, условия, итерации
и обработка исключений, использоваться не могут.

Вы можете замечательно писать код на Python без использования lambda. 

Однако вы наверняка натолкнетесь на них в написанной кем-то программе, в которой используется множество маленьких функций для вычисления результатов выражений, или же в программе, которая требует от пользователей предоставлять
функции обратного вызова (callback functions).

# 7.7.Захват переменных в анонимных функциях

Вы определили анонимную функцию, используя lambda, но вы также хотите захватить значения некоторых переменных во время определения.

Рассмотрим поведение следующей программы:

In [1]:
x = 10
a = lambda y: x + y
x = 20
b = lambda y: x + y

А теперь задайте себе вопрос: какими будут значения a(10) и b(10)? 

Если вы думаете, что 20 и 30, то ошибаетесь:


In [2]:
a(10)
#30

30

In [3]:
b(10)
#30

30

Проблема в том, что значение x, используемое в lambda-выражении, является свободной переменной, которая связывается во время выполнения, а не во время
определения. 

Так что значение x в lambda-выражениях будет таким, каким ему случится быть во время выполнения. 

Например:


In [4]:
x = 15
a(10)
#25

25

In [5]:
x = 3
a(10)
#13

13

Если же вы хотите, чтобы анонимная функция захватывала значение во время определения и сохраняла его, используйте значение по умолчанию:

In [6]:
x = 10
a = lambda y, x=x: x + y
x = 20
b = lambda y, x=x: x + y
a(10)
#20

20

In [7]:
b(10)
#30

30

Проблема, которую мы рассматриваем в этом рецепте, возникает, когда случается перемудрить с lambda-функциями. 

Например, вы можете ожидать, что при создании списка lambda-выражений с использованием генератора списка или цикла lambda-функции запомнят итерационные переменные во время определения.

Например:

In [8]:
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))


4
4
4
4
4


In [9]:
funcs = (lambda x: x+n for n in range(5))
for f in funcs:
    print(f(0))

0
1
2
3
4


Обратите внимание, что все функции считают, что n имеет последнее значение, полученное в ходе итераций. 

Теперь сравните со следующим:

In [10]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))


0
1
2
3
4


Как вы видите, теперь функции захватывают значения n во время определения.

#  7.8.Заставляем вызываемый объект с N аргументами работать так же, как вызываемый объект с меньшим количеством аргументов

У вас есть вызываемый объект, который вы хотели бы использовать в какой-то программе Python – возможно, в  качестве функции обратного вызова (callback function) или обработчика (handler), но он принимает слишком много аргументов и при вызове возбуждает исключение.

Если вам нужно уменьшить количество аргументов функции, используйте functools.partial(). 

Функция partial() позволяет присваивать фиксированные значения одному или более аргументам, что уменьшает количество аргументов, которые должны быть переданы в последующих вызовах. 

Например, у вас есть вот такая функция:

In [1]:
def spam(a, b, c, d):
    print(a, b, c, d)

А теперь попробуем partial(), чтобы зафиксировать значения некоторых аргументов:

In [2]:
from functools import partial
s1 = partial(spam, 1) # a = 1
s1(2, 3, 4)
#1 2 3 4

1 2 3 4


In [3]:
s1(4, 5, 6)
#1 4 5 6

1 4 5 6


In [4]:
s2 = partial(spam, d=42) # d = 42
s2(1, 2, 3)
#1 2 3 42

1 2 3 42


In [5]:
s2(4, 5, 5)
#4 5 5 42

4 5 5 42


In [7]:
s3 = partial(spam, 88, 99, d=42) # a = 88, b = 99, d = 42
s3(3)
#88 99 3 42

88 99 3 42


In [8]:
s3(4)
#88 99 4 42

88 99 4 42


In [9]:
s3(5)
#88 99 5 42

88 99 5 42


Понаблюдайте, как partial() фиксирует значения некоторых аргументов и возвращает новый вызываемый объект. 

Этот новый вызываемый объект принимает
пока еще не получившие значения аргументы, объединяя их с аргументами, переданными в partial(), и передает все в изначальную функцию.

Этот рецепт на самом деле связан с решением задачи обеспечения совместной работы, казалось бы, несовместимого кода. 

Проиллюстрируем это серией примеров.

Первый пример. 

Предположим, что у  вас есть список точек, представленных как кортежи координат (x, y). 

Вы можете использовать такую функцию для вычисления расстояния между двумя точками:

In [10]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]
import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

In [12]:
dist = partial(distance, points[0])
print('Растояние между фиксированной точкой (1,2) и точкой (4,6) равно ', dist((4,6)))


Растояние между фиксированной точкой (1,2) и точкой (4,6) равно  5.0


In [14]:
dist = partial(distance, points[1])
print('Растояние между фиксированной точкой (1,2) и точкой (3,5) равно ', dist((3,5)))

Растояние между фиксированной точкой (1,2) и точкой (3,5) равно  1.0


In [15]:
points1 = [ (1, 2), (2, 5), (4, 7), (11, 20), (6, 7), (12, 13), (16, 18), (15, 17) ]

In [17]:
#найти 5 ближайших точек к заданной
# pt - заданная точка
pt = (4, 3)
points1.sort(key=partial(distance,pt))
points1[:5]

[(2, 5), (1, 2), (4, 7), (6, 7), (12, 13)]

А теперь предположим, что вы хотите отсортировать все точки по их расстоянию до какой-то другой точки. 

Метод списков sort() принимает аргумент key,
который может быть использован для настройки поиска, но он работает только с функциями, которые принимают один аргумент (то есть distance() не подходит).

Вот как вы можете использовать partial(), чтобы решить данную проблему:

In [18]:
pt = (4, 3)
points.sort(key=partial(distance,pt))
points
#[(3, 4), (1, 2), (5, 6), (7, 8)]

[(3, 4), (1, 2), (5, 6), (7, 8)]

Развивая эту идею, заметим, что partial() часто может использоваться для настройки сигнатур аргументов функций обратного вызова, используемых в других
библиотеках. 

Например, вот пример кода, который использует multiprocessing для асинхронного вычисления результата, который передается функции обратного вызова, принимающей результат и необязательный аргумент настройки логирования:

In [19]:
def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)

In [ ]:
# Функция-пример
def add(x, y):
    return x + y


import logging
from multiprocessing import Pool
from functools import partial
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger('test')
p = Pool()
p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
p.close()
p.join()

При передаче функции обратного вызова с использованием apply_async() дополнительный аргумент настройки логирования передается с  использованием partial(). 

А  multiprocessing просто вызывает функцию обратного вызова с  единственным значением.

В качестве похожего примера рассмотрим задачу написания сетевых серверов.

Модуль socketserver позволяет сделать это без особого труда. 

Например, вот простой эхо-сервер:

In [ ]:
from socketserver import StreamRequestHandler, TCPServer
class EchoHandler(StreamRequestHandler):
    def handle(self):
        for line in self.rfile:
            self.wfile.write(b'GOT:' + line)
            
serv = TCPServer(('', 15000), EchoHandler)
serv.serve_forever()

Предположим, что вы хотите наделить класс EchoHandler методом __init__(), который принимает дополнительный конфигурирующий аргумент. 

Например:

In [ ]:
class EchoHandler(StreamRequestHandler):
    # ack – это добавленный обязательный именованный аргумент.
    # *args, **kwargs – это любые обычные предоставленные параметры
    # (которые переданы)
    def __init__(self, *args, ack, **kwargs):
        self.ack = ack
        super().__init__(*args, **kwargs)
    def handle(self):
        for line in self.rfile:
            self.wfile.write(self.ack + line)


Если вы внесете это изменение, то обнаружите, что больше нет очевидного пути вставить его в класс TCPServer. 

На самом деле вы обнаружите, что код начал возбуждать такие исключения:

Exception happened during processing of request from ('127.0.0.1', 59834)
Traceback (most recent call last):
 ...
TypeError: __init__() missing 1 required keyword-only argument: 'ack'

На первый взгляд кажется невозможным исправить этот код без попыток поправить исходник socketserver или еще какого-то странного обходного решения.

Однако задача легко решается с помощью partial() – используйте ее, чтобы предоставить значение аргумента ack:

In [ ]:
from functools import partial
serv = TCPServer(('', 15000), partial(EchoHandler, ack=b'RECEIVED:'))
serv.serve_forever()

В этом примере определение аргумента ack в методе __init__() может показаться немного странным, но он определяется как обязательный именованный аргумент. 

Это подробно рассматривается в рецепте 7.2.
Иногда функциональность partial() заменяется lambda-выражением. 

Например, в предыдущем примере можно применить такие инструкции:

In [ ]:
points.sort(key=lambda p: distance(pt, p))
p.apply_async(add, (3, 4), callback=lambda result: output_result(result,log))
serv = TCPServer(('', 15000), 
                 lambda *args, **kwargs: EchoHandler(*args, 
                                                     ack=b'RECEIVED:', 
                                                     **kwargs))

Этот код работает, но он более многословен и может запутать того, кто его читает. 

Использование partial() более явно сообщает о вашем намерении (передать значения некоторым аргументам).

# 7.9. Замена классов с одним методом функциями

У вас есть класс, который определяет только один метод, кроме __init__(). 

Однако для упрощения вашего кода вы бы хотели заменить его на простую функцию.

Во многих случаях классы с одним методом могут быть превращены в функции с помощью замыканий. 

Рассмотрите, например, следующий класс, который позволяет пользователю загружать страницы по URL с использованием некой шаблонной схемы:

In [4]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))
    
# Пример использования. Скачать данные об акциях с Yahoo
yahoo = UrlTemplate('https://query1.finance.yahoo.com/v10/finance/quoteSummary/{ticker}?modules={modules}')
for line in yahoo.open(ticker='AAPL', modules='assetProfile'):
    print(line.decode('utf-8'))

{"quoteSummary":{"result":[{"assetProfile":{"address1":"One Apple Park Way","city":"Cupertino","state":"CA","zip":"95014","country":"United States","phone":"408-996-1010","website":"http://www.apple.com","industry":"Consumer Electronics","sector":"Technology","longBusinessSummary":"Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides digital content stores and streaming services; AppleCare support services; and iCloud, a cloud service, which stores music, photos, contacts, calendars, mail, documents, and others. In addition, the company offers various service, such as A

Этот класс может быть заменен намного более простой функцией:

In [5]:
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# Пример использования
yahoo = urltemplate('https://query1.finance.yahoo.com/v10/finance/quoteSummary/{ticker}?modules={modules}')
for line in yahoo(ticker='AAPL', modules='assetProfile'):
    print(line.decode('utf-8'))


{"quoteSummary":{"result":[{"assetProfile":{"address1":"One Apple Park Way","city":"Cupertino","state":"CA","zip":"95014","country":"United States","phone":"408-996-1010","website":"http://www.apple.com","industry":"Consumer Electronics","sector":"Technology","longBusinessSummary":"Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides digital content stores and streaming services; AppleCare support services; and iCloud, a cloud service, which stores music, photos, contacts, calendars, mail, documents, and others. In addition, the company offers various service, such as A

Во многих случаях единственной причиной использовать класс с одним методом является необходимость сохранять дополнительное состояние для использования в методе. 

Например, единственное назначение класса UrlTemplate заключается в сохранении значения template, чтобы оно могло быть использовано в методе open().

Использование вложенной функции (замыкания), как показано выше, часто будет намного более элегантным решением. 

Замыкание – это просто функция, но с  дополнительным окружением переменных, которые используются внутри
функции. 

Ключевое преимущество замыкания в том, что оно запоминает окружение, в котором было определено. 

Поэтому в примере функция opener() запоминает значение аргумента template() и использует его в последующих вызовах.

Всякий раз, когда вы пишете код и встречаетесь с задачей прикрепления дополнительного состояния к функции, вспоминайте о замыканиях. 

Они часто являются более минималистичным и элегантным решением, нежели альтернатива (превращение функции в полноценный класс).

# 7.10. Передача дополнительного состояния с функциями обратного вызова

Вы пишете код, который опирается на использование функций обратного вызова (например, на обработчики событий, функции обратного вызова на завершения
и т. п.), но вы хотите получить функцию обратного вызова, переносящую дополнительное состояние для использования в функции обратного вызова.

Этот рецепт относится к способу использования функций обратного вызова, который можно обнаружить во многих библиотеках и фреймворках – особенно тех, которые связаны с асинхронной обработкой. 

Рассмотрим следующую функцию, которая вызывает функцию обратного вызова:

In [1]:
def apply_async(func, args, *, callback):
 # Вычислить результат
 result = func(*args)
    
 # Вызвать функцию обратного вызова с результатом
 callback(result)

В реальной жизни такой код может выполнять различные типы продвинутой обработки, включающей потоки, процессы и таймеры, но в данном случае это не
главное. 

Мы просто сосредоточимся на вызове функции обратного вызова. 

Вот пример использования приведенного выше кода:


In [2]:
def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)
#Got: 5


Got: 5


In [3]:
apply_async(add, ('hello', 'world'), callback=print_result)
#Got: helloworld

Got: helloworld


Как вы можете видеть, функция print_result() принимает только один аргумент, который представляет собой результат. 

Никакая другая информация не передается. 

Это отсутствие информации иногда может представлять собой проблему, когда вы хотите, чтобы функция обратного вызова взаимодействовала с другими переменными или частями окружения.

Способ передать дополнительную информацию в функцию обратного вызова – это использование связанного метода вместо простой функции. 

Например, этот класс хранит внутренний последовательный номер, который инкрементально
увеличивается каждый раз, когда получен результат:


In [4]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
        
    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

Чтобы использовать этот класс, вы могли бы создать экземпляр и использовать связанный метод handler в качестве функции обратного вызова:


In [5]:
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
#[1] Got: 5

[1] Got: 5


In [6]:
apply_async(add, ('hello', 'world'), callback=r.handler)
#[2] Got: helloworld

[2] Got: helloworld


В качестве альтернативы классу вы также можете использовать для хранения состояния замыкание:

In [9]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

Вот пример использования такого варианта:

In [10]:
handler = make_handler()
apply_async(add, (2, 3), callback=handler)
#[1] Got: 5

[1] Got: 5


In [11]:
apply_async(add, ('hello', 'world'), callback=handler)
#[2] Got: helloworld

[2] Got: helloworld


В качестве еще одной вариации на эту тему вы также иногда можете использовать корутину (coroutine (сопрограмма)) для выполнения той же задачи:


In [13]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))


Для корутины вы можете использовать метод send() в  качестве функции обратного вызова:

In [14]:
handler = make_handler()
next(handler) # Продвигаемся к yield
apply_async(add, (2, 3), callback=handler.send)
#[1] Got: 5

[1] Got: 5


In [15]:
apply_async(add, ('hello', 'world'), callback=handler.send)
#[2] Got: helloworld

[2] Got: helloworld


И последнее: вы также можете передать состояние в функцию обратного вызова, используя дополнительный аргумент и применяя функцию partial(). 

Например:

In [16]:
class SequenceNo:
    def __init__(self):
        self.sequence = 0

def handler(result, seq):
    seq.sequence += 1
    print('[{}] Got: {}'.format(seq.sequence, result))
    
seq = SequenceNo()
from functools import partial
apply_async(add, (2, 3), callback=partial(handler, seq=seq))
#[1] Got: 5

[1] Got: 5


In [17]:
apply_async(add, ('hello', 'world'), callback=partial(handler, seq=seq))
#[2] Got: helloworld

[2] Got: helloworld


Программы, основанные на функциях обратного вызова, часто подвержены риску превратиться в огромную беспорядочную кучу. 

Частично эта проблема возникает, потому что функция обратного вызова нередко отсоединена от кода, который делает первоначальный запрос, приводящий к ее выполнению. 

Поэтому окружение выполнения между созданием запроса и обработкой результата теряется. 

Если вы хотите продолжить функцию обратного вызова в процедуре из нескольких шагов, вам нужно понять, как сохранить и восстановить ассоциированное состояние.

Существует два основных подхода, которые полезны для захвата и  переноса состояния.

Вы можете переносить его в экземпляре (например, прикрепленном к связанному методу), или же вы можете переносить его в замыкании (вложенной функции). 

Из этих двух приемов замыкания, вероятно, немного более легковесны и естественны, поскольку просто создаются из функций. 

Они также автоматически захватывают все использованные переменные. 

Это освобождает вас от необходимости беспокоиться по поводу того, какое именно состояние нужно сохранить (это автоматически определяется вашим кодом).

При использовании замыканий вам нужно осторожно обращаться с изменяемыми (мутабельными) переменными. 

В вышеприведенном решении объявление nonlocal используется для обозначения того, что переменная sequence изменяется изнутри функции обратного вызова. 

Без этого объявления вы бы получили ошибку.

Использование корутины (сопрограммы) в  качестве обработчика функций обратного вызова интересно тем, что это тесно связано с подходом с использованием замыканий. 

В  некотором смысле он даже прозрачнее, поскольку представляет собой одну функцию. 

Более того, переменные можно свободно изменять и  не беспокоиться об объявлениях nonlocal. 

Потенциальный недостаток в  том, что корутины не так легко понять, как другие компоненты Python. 

Есть также несколько тонких моментов – таких как необходимость вызывать next() на корутине,
перед тем как ее использовать. 

Тем не менее корутины можно использовать и подругому – например, для определения встроенной функции обратного вызова (см.следующий рецепт).

Последний прием с использованием partial() полезен, если вам нужно просто передать дополнительные значения в функцию обратного вызова. 

Иногда вместо partial() мы можем достичь того же с помощью lambda:


In [ ]:
apply_async(add, (2, 3), callback=lambda r: handler(r, seq))
#[1] Got: 5

Дополнительные примеры мы можете найти в рецепте 7.8, где показывается использование partial() для изменения аргументных сигнатур.

# 7.11. Встроенные функции обратного вызова

Вы пишете код, в котором используются функции обратного вызова, но вас беспокоит быстрое размножение маленьких функций и излишняя сложность потока
управления. 

Вы бы хотели как-то заставить код выглядеть более похожим на нормальную последовательность процедурных шагов.

Функции обратного вызова могут быть встроены в  функцию путем использования генераторов и  корутин (сопрограмм). 

Предположим, у  вас есть функция,
которая выполняет какую-то работу и вызывает функцию обратного вызова (см.
рецепт 7.10):

In [1]:
def apply_async(func, args, *, callback):
 # Вычисляем результат
 result = func(*args)
 # Вызываем функцию обратного вызова с результатом
 callback(result)

Теперь взгляните на поддерживающий код, который использует класс Async и декоратор inlined_async:

In [2]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args
        
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

Эти два фрагмента кода позволят вам встроить в строку шаги функции обратного вызова, используя инструкции yield. 

Например:

In [3]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')


In [4]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


Если вы вызовете test(), то получите такой вывод:

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye

Если исключить специальный декоратор и использование yield, то вы заметите, что функции обратного вызова нигде не появляются (только «под капотом»).

Этот рецепт – испытание для ваших знаний в области функций обратного вызова, генераторов и потока управления.

Во-первых, основная фишка кода с  функциями обратного вызова в том, что текущее вычисление приостанавливается и  возобновляется в  какой-то момент времени позже (т. е. асинхронно). 

Когда вычисление возобновляется, для продолжения обработки выполняется функция обратного вызова. 

Функция apply_sync() иллюстрирует важнейшие составляющие выполнения функции обратного вызова, хотя в реальном мире процесс может быть намного сложнее (в нем могут использоваться потоки, процессы, обработчики событий и т. п.).

Идея того, что вычисление приостановится и возобновится, естественным образом отображается на модель выполнения генератора. Если точнее, то операция
yield заставляет генератор выдавать значение и приостанавливаться. 

Последующие вызовы методов генератора __next__() или send() заставят его снова запуститься.

Имея это в виду, мы можем понять, что суть этого рецепта заключена в декораторе inline_async(). 

Главная идея в том, что декоратор пошагово проводит генератор через все его инструкции yield. 

Чтобы это сделать, создается очередь результатов и  изначально наполняется значениями None. 

Затем инициируется цикл,в котором результат вынимается из очереди и посылается в генератор. 

Это вызывает следующий yield, где принимается экземпляр Async. 

Затем цикл смотрит на функцию и аргументы и вызывает асинхронное вычисление apply_sync(). 

Однако наиболее хитрая часть этого вычисления в том, что вместо использования обычной функции обратного вызова функция обратного вызова установлена на метод
очереди put().

В этот момент остается открытым вопрос о том, что произойдет. 

Главный цикл немедленно возвращается наверх и просто выполняет операцию get() на очереди. 

Если данные присутствуют, то это должен быть результат, помещенный туда функцией обратного вызова put(). 

Если же ничего нет, операция блокируется
и ждет, когда придет результат. 

Как это может произойти – зависит от конкретной
реализации функции apply_async().

Если вы сомневаетесь, что такая безумная штука может работать, вы можете попробовать ее с библиотекой multiprocessing и заставить асинхронные операции
выполняться в отдельных процессах:

In [ ]:
import multiprocessing
pool = multiprocessing.Pool()
apply_async = pool.apply_async
# Запускаем тестовую функцию
test()

Вы обнаружите, что это работает, но чтобы разобраться в потоке управления, вам потребуется немало кофе.

Прием скрытия нетривиального потока управления за генераторами можно найти повсеместно в  стандартной библиотеке и  сторонних пакетах. 

Например, декоратор @contextmanager из библиотеки contextlib выполняет похожий безумный фокус, который через инструкцию yield склеивает вход в менеджер контекста и выход из него. 

Популярный пакет Twisted тоже использует похожие встроенные функции обратного вызова.

# 7.12. Доступ к переменным, определенным внутри замыкания

Вы хотите добавить в  замыкание функции, которые позволят получать доступ и изменять внутренние переменные.

В обычном случае внутренние переменные замыкания полностью скрыты от внешнего мира. 

Однако вы можете предоставить доступ путем написания функций для доступа и прикрепления их к замыканию в качестве атрибутов функции.

Например:

In [10]:
def sample():
    n = 0
    # Функция-замыкание
    def func():
        print('n = ', n)
        
    #Методы доступа к n
    def get_n():
        return n
    
    def set_n(value):
        nonlocal n
        n = value
        
    # Прикрепление в качестве атрибутов функции
    func.get_n = get_n
    func.set_n = set_n
    return func

Вот пример использования этого кода:

In [12]:
f = sample()
f()
#n = 0

n =  0


In [11]:
f.set_n(10)
f()
#n = 10

n =  10


In [9]:
f.get_n()
#10

10

Две главные возможности языка позволяют этому рецепту работать. 

Во-первых, инструкции nonlocal делают возможным написание функций, которые изменяют внутренние  переменные. 

Во-вторых, атрибуты функции дают возможность напрямую прикреплять методы для доступа к замыканию, и они работают практически так же, как методы экземпляра (хотя классы тут не используются).

Небольшое дополнение к  этому рецепту позволит замыканиям эмулировать экземпляры класса. 

Все, что вам нужно, – это скопировать внутренние функции в словарь экземпляра и возвратить его. 

Например:

In [13]:
import sys

class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals
            
        # Обновить словарь экземпляра вызываемыми объектами
        self.__dict__.update((key,value) for key, value in locals.items()
                             if callable(value) )
        # перегружаем специальные методы
    def __len__(self):
        return self.__dict__['__len__']()

In [14]:
# Пример использования
def Stack():
    items = []
    
    def push(item):
        items.append(item)

    def pop():
        return items.pop()

    def __len__():
        return len(items)
    return ClosureInstance()

Вот интерактивный сеанс, который показывает, как все это работает:

In [15]:
s = Stack()
s
#<__main__.ClosureInstance object at 0x10069ed10>

In [16]:
s.push(10)
s.push(20)
s.push('Hello')
len(s)
#3

3

In [17]:
s.pop()
#'Hello'

'Hello'

In [18]:
s.pop()
#20

20

In [19]:
s.pop()
#10

10

Интересно, что этот код работает немного быстрее аналога, использующего обычное определение класса. 

Например, вы можете проверить производительность по сравнению с таким классом:

In [20]:
class Stack2:
    def __init__(self):
        self.items = []
        
    def push(self, item):
        self.items.append(item)
        
    def pop(self):
        return self.items.pop()
    
    def __len__(self):
        return len(self.items)

Если вы это сделаете, то получите похожие результаты:

In [21]:
from timeit import timeit
# Тест с использованием замыканий
s = Stack()
timeit('s.push(1);s.pop()', 'from __main__ import s')
#0.9874754269840196

0.8244574000000284

In [22]:
# Тест с использованием класса
s = Stack2()
timeit('s.push(1);s.pop()', 'from __main__ import s')
#1.0707052160287276

0.8457025000002432

Как показано выше, версия на базе замыкания работает на 8 % быстрее. 

По большей части выигрыш возникает за счет прямого доступа к переменным экземпляра. 

Замыкания быстрее, потому что не используют дополнительную переменную self.

Рэймонд Хеттингер предложил еще более дьявольский вариант этой идеи. 

Однако если вы склоняетесь использовать что-то такое в своей программе, помните, что это просто диковинная замена настоящему классу. 

Например, ключевые возможности типа наследования, свойств, дескрипторов или методов класса работать не будут. 

Вам также придется поплясать с бубном, чтобы заставить специальные методы работать (например, обратите внимание на реализацию метода __len__() в ClosureInstance).

И последнее: вы рискуете затруднить жизнь людям, которые будут читать ваш код и размышлять о том, почему в нем нет нормального определения класса (конечно, они также задумаются, почему ваша версия работает быстрее). 

Но, в любом случае, это интересный пример того, чего можно достичь, предоставляя доступ
к «внутренностям» замыкания.

Добавление методов в замыкания может иметь больше смысла в задачах, в рамках которых вам нужно делать вещи типа сброса внутреннего состояния, сброса
буферов, очистки кеша или реализации какого-то механизма обратной связи.